# Import thư viện

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from requests import get
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback

In [3]:
# Mở trình duyệt Google Chrome ở trang thái Tab ẩn danh
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
s = Service('C:\chromedriver')
driver = webdriver.Chrome(service=s, options=options)
driver.implicitly_wait(0)

In [4]:
# List tất cả url movie
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

In [5]:
df_movie = []

for url_movie in list_all_url_movie[:5]:
    # Mở url phim
    driver.get(url_movie)
    
    try:
        # Đợi cho đến khi url load xong
        element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
        soup = BeautifulSoup(html_of_interest, 'lxml')

        sleep(2)

        ## Tạo Dict chứa thông tin về phim
        dict_info_movie = {}

        # Tiêu đề phim
        dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text

        # Điểm đánh giá của nhà phê bình
        dict_info_movie['Tomatometer score'] = soup.select('score-board')[0].attrs['tomatometerscore']

        # Điểm đánh giá của khán giả
        dict_info_movie['Audience score'] = soup.select('score-board')[0].attrs['audiencescore']

        # Số lượt đánh giá của nhà phê bình
        dict_info_movie['Tomatometer count'] = soup.select('a[slot="critics-count"]')[0].text.strip()

        # Số lượt đánh giá của khán giả
        dict_info_movie['Audience count'] = soup.select('a[slot="audience-count"]')[0].text.strip()

        # Trạng thái đánh giá phim của nhà phê bình
        dict_info_movie['Tomatometer state'] = soup.select('score-board')[0].attrs['tomatometerstate']

        # Trạng thái đánh giá phim của khán giả
        dict_info_movie['Audience state'] = soup.select('score-board')[0].attrs['audiencestate']

        # Thông tin phim
        list_name_info = soup.select('div[class="meta-label subtle"]')
        list_info = soup.select('div[data-qa="movie-info-item-value"]')

        for num_info in range(len(list_name_info)):
            name_info = list_name_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            info = list_info[num_info].text.replace('  ', '').replace('\n', '').replace(':', '')
            dict_info_movie[name_info] = info

        # Tóm tắt nội dụng phim
        dict_info_movie['Synopsis'] = soup.select('div[id="movieSynopsis"]')[0].text.strip()

        # Diễn viên
        list_raw_cast = soup.select('span[class="characters subtle smaller"]')
        list_cast = []

        for c in list_raw_cast:
            list_cast.append(c.attrs['title'])

        dict_info_movie['Cast'] = ', '.join(list_cast)

        df_movie.append(dict_info_movie)

        # Reviews của nhà phê bình
        driver.get(url_movie + '/reviews')

        element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="layout reviews-page-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
        soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

        # Check có review của nhà phê bình không ?
        check_critic_review = soup_critic_review.select('div[class="review_table"]')[0].text.strip()

        if check_critic_review != '':
            list_critic_review = []
            list_top_critic_review = []
            
            # Số lượng review trong 1 trang
            num_rv = len(soup_critic_review.select('div[class="review-row"]'))

            for num in range(num_rv):
                rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text
                list_critic_review.append(rv)

                # Check top critic reviews
                check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                if check_top_critic != []:
                    list_top_critic_review.append(rv)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_critic_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="review_table"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_critic_review)
                    soup_critic_review = BeautifulSoup(html_of_interest, 'lxml')

                    num_rv = len(soup_critic_review.select('div[class="review-row"]'))

                    for num in range(num_rv):
                        rv = soup_critic_review.select('div[class="review-row"]')[num].find_all('p', {'class': 'review-text'})[0].text.strip()
                        list_critic_review.append(rv)

                        check_top_critic = soup_critic_review.select('div[class="review-row"]')[num].find_all('rt-icon-top-critic')

                        if check_top_critic != []:
                            list_top_critic_review.append(rv)
                    
                except:
                    break
            dict_info_movie['Critic review'] = '<>'.join(list_critic_review)
            if list_top_critic_review != []:
                dict_info_movie['Top critic review'] = '<>'.join(list_top_critic_review)
            else:
                dict_info_movie['Top critic review'] = 'No top critic reviews'
        elif check_critic_review == '':
            dict_info_movie['Critic review'] = 'No critic reviews'
            dict_info_movie['Top critic review'] = 'No top critic reviews'

        # Reviews của khán giả
        driver.get(url_movie + '/reviews?type=user')

        element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-qa="reviews-container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
        soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

        ## Check có review của khán giả không ?
        check_audience_review = soup_audience_review.select('div[id="movieUserReviewsContent"]')

        if len(check_audience_review) != 0:
            list_audience_review = []
            
            for rv in soup_audience_review.select('p[data-qa="review-text"]'):
                list_audience_review.append(rv.text)
            
            while True:
                sleep(1)
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, 'rt-button[class="js-prev-next-paging-next"]')
                    driver.execute_script("arguments[0].click();", next_button)
                    
                    element_audience_review = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="movieUserReviewsContent"]')))
                    html_of_interest = driver.execute_script('return arguments[0].innerHTML',element_audience_review)
                    soup_audience_review = BeautifulSoup(html_of_interest, 'lxml')

                    for rv in soup_audience_review.select('p[data-qa="review-text"]'):
                        list_audience_review.append(rv.text)
                    
                except:
                    break
            dict_info_movie['Audience review'] = '<>'.join(list_audience_review)
        else:
            dict_info_movie['Audience review'] = 'Không có review của khán giả'

    except Exception:
        traceback.print_exc()
        print(url_movie)
        continue
    
df_movie = pd.DataFrame(df_movie)

In [7]:
df_movie

,Title,Tomatometer score,Audience score,Tomatometer count,Audience count,Tomatometer state,Audience state,Genre,Original Language,Director,...,Runtime,Synopsis,Cast,Critic review,Top critic review,Audience review,Producer,Release Date (Theaters),Box Office (Gross USA),Distributor
0,Beg!,,,0 Reviews,50+ Ratings,,,Horror,English,Robert Golden,...,1h 48m,Murder and mayhem unfold as two rivals jockey ...,"Peta Lily, Philip Pellew, Julian Bleach, Olega...",No critic reviews,No top critic reviews,I'm not really sure what to say about this. I...,NaN,NaN,NaN,NaN
1,Limo,,,0 Reviews,0 Ratings,,,Adventure,Dutch,Guy Goossens,...,1h 36m,Four girls visit a mysterious village.,"Jan Decleir, Jack Wouterse, Jaap Spijkers, Ing...",No critic reviews,No top critic reviews,Không có review của khán giả,"Guy Goossens, Mark Punt",NaN,NaN,NaN
2,Tejano,,71,4 Reviews,Fewer than 50 Ratings,,upright,Mystery & thriller,English,David Blue Garcia,...,1h 28m,A farmhand in South Texas breaks his arm to sm...,"Patrick Mackie, Roland Uribe, Mayra Leal, Adri...",The story of a South Texas ranch hand who gets...,An entertaining blast of Texas noir that nods ...,The movie is a good western and David is a goo...,David Blue Garcia,NaN,NaN,NaN
3,Outpatient,,,0 Reviews,0 Ratings,,,Drama,English,Alec Carlen,...,1h 45m,A doctor (Catherine Kellner) and a publisher t...,"Justin Kirk, Catherine Kellner, Claudia Mason,...",No critic reviews,No top critic reviews,Không có review của khán giả,NaN,NaN,NaN,NaN
4,Paa,50,67,8 Reviews,"1,000+ Ratings",rotten,upright,Drama,Hindi,R. Balki,...,2h 13m,A politician's 13-year-old son has a rare diso...,"Amitabh Bachchan, Abhishek Bachchan, Vidya Bal...","The film, which peters off into vague sub-plot...","Disappointingly, Paa is not as out-of-the-box ...",What a refreshing experience<>Fantastic movie....,Sunil Manchanda,"Dec 4, 2009 limited",$199.2K,Big Pictures


In [8]:
list_all_url_movie[:5]

['https://www.rottentomatoes.com/m/beg1994',
 'https://www.rottentomatoes.com/m/limo',
 'https://www.rottentomatoes.com/m/tejano',
 'https://www.rottentomatoes.com/m/outpatient',
 'https://www.rottentomatoes.com/m/1221483-paa']

In [6]:
# df_movie.to_csv('Crawl_test_1000_movie.csv', index=False)